In [1]:
import lxml.html
import requests
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import re

C:\Users\nyker\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
target_url = "http://baseball-freak.com/game/0.html"
target_html = requests.get(target_url).text
parser = lxml.html.HTMLParser(encoding='utf-8')
tree = lxml.html.parse(target_url,parser)

urls = []
years = range(2009,2015)
for item in years:
    url = "http://baseball-freak.com/game/"+str(item)[-2:]+"/0.html"
    urls.append(url)
years.append(2015)
urls.append("http://baseball-freak.com/game/0.html")
print urls,years

['http://baseball-freak.com/game/09/0.html', 'http://baseball-freak.com/game/10/0.html', 'http://baseball-freak.com/game/11/0.html', 'http://baseball-freak.com/game/12/0.html', 'http://baseball-freak.com/game/13/0.html', 'http://baseball-freak.com/game/14/0.html', 'http://baseball-freak.com/game/0.html'] [2009, 2010, 2011, 2012, 2013, 2014, 2015]


In [11]:
name_list = [
    ["c","広"],
    ["g","巨"],
    ["t","阪"],
    ["d","中"],
    ["yb","横"],
    ["s","ヤ"]
]

#get player infomation df
def get_playerinfo_df():
    contents = []
    header = ["Team"]
    for i,names in enumerate(name_list):
        url1 = "http://baseball-data.com/player/"+names[0]+"/"
        teamname =names[1]
        tree1 = lxml.html.parse(url1,parser)
        if i == 0:
            for item in tree1.xpath("//table/thead/tr/th"):
                header.append(item.text_content())

        for tr_item in tree1.xpath("//table/tbody/tr"):
            data = [teamname]
            for td_item in tr_item.getchildren():
                val = td_item.text_content()
                if val[-2:] == u"万円" or val[-2:]=="kg" or val[-2:]=="cm":
                    val = val[:-2]
                elif val[-1:]==u"歳" or val[-1:]==u"年":
                    val = val[:-1]
                elif val[-1:]==u"型":
                    val = val[:-1]

                data.append(val)
            contents.append(data)
    mydf = pd.DataFrame(contents)
    mydf.columns = header
    return mydf

In [12]:
pInfo_df = get_playerinfo_df()
pInfo_df[u"年俸(推定)"] = pInfo_df[u"年俸(推定)"].str.replace(',','')
pInfo_df[u"投"] = pInfo_df[u"投打"].str[0]
pInfo_df[u"打"] = pInfo_df[u"投打"].str[1]
del pInfo_df[u"投打"]
pInfo_df.to_csv("player_info_table.csv",encoding="utf-8")

In [13]:
# pInfo_df[u"投"] = pInfo_df[u"投打"].str[0]
# pInfo_df[u"打"] = pInfo_df[u"投打"].str[1]
pInfo_df = pInfo_df.convert_objects(convert_timedeltas=True)
pInfo_df = pInfo_df.convert_objects(convert_numeric=True)
pInfo_df.dtypes

C:\Users\nyker\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
C:\Users\nyker\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


Team      object
No.        int64
選手名       object
守備        object
生年月日      object
年齢         int64
年数         int64
身長         int64
体重         int64
血液型       object
出身地       object
年俸(推定)     int64
投         object
打         object
dtype: object

In [14]:
pInfo_df

,Team,No.,選手名,守備,生年月日,年齢,年数,身長,体重,血液型,出身地,年俸(推定),投,打
0,広,0,上本 崇司,内野手,1990/08/22,25,4,170,71,A,広島,880,右,両
1,広,0,中東 直己,外野手,1981/10/05,34,10,168,73,B,広島,1300,右,左
2,広,2,田中 広輔,内野手,1989/07/03,26,3,171,81,A,神奈川,4100,右,左
3,広,4,小窪 哲也,内野手,1985/04/12,30,9,175,80,O,奈良,4000,右,右
4,広,5,ルナ,内野手,1980/02/01,36,4,187,99,B,ドミニカ共和国,12000,右,右
5,広,6,梵 英心,内野手,1980/10/11,35,11,173,76,B,広島,9000,右,右
6,広,7,堂林 翔太,内野手,1991/08/17,24,7,183,83,A,愛知,1830,右,右
7,広,9,丸 佳浩,外野手,1989/04/11,26,9,177,90,AB,千葉,8500,右,左
8,広,10,岩本 貴裕,外野手,1986/04/18,29,8,182,96,A,広島,1500,左,左
9,広,11,福井 優也,投手,1988/02/08,28,6,178,80,A,岡山,4200,右,右


In [15]:
#batting data

def get_hitter_or_pitcher_stats_df(Ispitcher):
    base_url = "http://baseball-data.com/stats/hitter-"
    if Ispitcher:      
        base_url = "http://baseball-data.com/stats/pitcher-"
    header = ["Team"]
    contents = []
    for i,names in enumerate(name_list):
        target_url = base_url + names[0] + "/"
        teamname = names[1]
        tree = lxml.html.parse(target_url,parser)
        if i == 0:
            for th in tree.xpath("//table[@class='tablesorter stats']/thead[1]/tr/th"):
                header.append(th.text_content())
        for tr in tree.xpath("//table[@class='tablesorter stats']/tbody/tr"):
            data = [teamname]
            for td in tr.getchildren():
                val = td.text_content()
                if val == u"-":
                    val = "NaN"
                data.append(val)
            contents.append(data)
    mydf = pd.DataFrame(contents)
    mydf.columns = header
    return mydf

In [16]:
batting2015df = get_hitter_or_pitcher_stats_df(Ispitcher=False)
batting2015df.to_csv("battingdata2015.csv",encoding="utf-8")

In [17]:
Ongamebatterdf = batting2015df.query('XR27 != \'NaN\'')
Ongamebatterdf=Ongamebatterdf.convert_objects(convert_numeric=True)
Ongamebatterdf[Ongamebatterdf[u"打数"]>50].sort(u"XR27",ascending=False).groupby("Team").sum()

ImportError: 'numexpr' is not installed or an unsupported version. Cannot use engine='numexpr' for query/eval if 'numexpr' is not installed

In [310]:
pitcher2015df = get_hitter_or_pitcher_stats_df(True)
pitcher2015df.to_csv("pitcher2015.csv",encoding="utf-8")

In [207]:
# matching dataの取得をします
urls = []
years = range(2009,2015)
for item in years:
    url = "http://baseball-freak.com/game/"+str(item)[-2:]+"/0.html"
    urls.append(url)
years.append(2015)
urls.append("http://baseball-freak.com/game/0.html")
print urls,years

contents = []
for y in range(len(years)):
    url = urls[y]
    year = years[y]
    tree = lxml.html.parse(url,parser)
    tr_items = tree.xpath('//table[@class="schedule"]/tr')    
    for tr_obj in tr_items:
        date = ""
        for i,td_obj in enumerate(tr_obj.getchildren()):
            if i == 0: #date
                datestring = td_obj.text_content()[:-3]
                date_list = datestring.encode("utf-8").strip("日").split("月")
                date = str(year)+"/"+date_list[0]+"/"+date_list[1]
                continue
            if len(td_obj.text_content()) == 1:
                continue
            team_p = td_obj.getchildren()
            text = team_p[0].text_content()
            text2 = team_p[1].text_content()
            data = [date,text[0],text[1:-1].split("-")[0].strip(" "),text[-1],text[1:-1].split("-")[-1].strip(),text2[:-5],text2[-5:]]
            contents.append(data)

In [242]:
machup_df = pd.DataFrame(contents)
machup_df.columns = ["Date","HomeTeam","HomePoint","AwayTeam","AwayPoint","Studium","StartTime"]
machup_df.index
machup_df.to_csv("match_up_table.csv",encoding="utf-8",index_label="Game_Index")

In [129]:
datestring = "3月28日"
datestring.strip("日").split("月")

['3', '28']

[2009, 2010, 2011, 2012, 2013, 2014]


In [218]:
chancelldf = machup_df[machup_df["HomePoint"] == u"中止"].groupby("Studium").size()
chancelldf.sort(ascending=False)
chancelldf.columns = ["chancell"]
print chancelldf

Studium
神宮        30
横浜        29
甲子園       29
マツダスタ     26
QVCマリン    23
コボスタ宮城    15
Ｋスタ宮城     12
Kスタ宮城     12
ほっと神戸      8
千葉マリン      5
西武ドーム      3
北九州        3
富山         3
秋田         2
平塚         2
静岡         2
スカイマーク     2
大宮         2
岐阜         2
旭川         2
金沢         2
帯広         1
福井         1
鹿児島        1
松山         1
熊本         1
函館         1
三次         1
釧路         1
那覇         1
長野         1
皇子山        1
盛岡         1
東京ドーム      1
dtype: int64


In [219]:
done_game_df = machup_df[machup_df["HomePoint"] != u"中止"]
done_game_df =done_game_df.convert_objects(convert_numeric=True)
done_game_df.plot(kind='hexbin', x='HomePoint', y='AwayPoint', gridsize=25)
plt.show()

In [204]:
done_game_df.groupby("Studium").mean().plot()
plt.show()

In [240]:
hinda = done_game_df.query("HomePoint + AwayPoint <= 2")
hinda.sort(ascending=False)

,Date,HomeTeam,HomePoint,AwayTeam,AwayPoint,Studium,StartTime
6260,2015/10/3,ロ,2,楽,0,QVCマリン,14:00
6236,2015/9/28,阪,2,巨,0,甲子園,18:00
6232,2015/9/27,オ,0,日,2,京セラD大阪,13:00
6219,2015/9/24,巨,2,広,0,東京ドーム,18:00
6217,2015/9/23,ロ,1,楽,0,QVCマリン,13:00
6199,2015/9/20,ソ,1,ロ,0,ヤフオクドーム,14:00
6197,2015/9/20,楽,2,オ,0,コボスタ宮城,13:00
6193,2015/9/19,ソ,2,ロ,0,ヤフオクドーム,18:00
6179,2015/9/16,オ,0,ソ,2,京セラD大阪,18:00
6173,2015/9/15,広,1,巨,0,マツダスタ,18:00


In [239]:
hinda

HomeTeam
中    67
阪    51
広    47
巨    41
ソ    39
オ    38
ロ    34
楽    33
ヤ    32
西    31
日    30
横    15
D     8
dtype: int64